<a href="https://colab.research.google.com/github/427paul/ai_agent/blob/main/ai_agent_04_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U "langchain==0.3.*" "langchain-core==0.3.*" "langchain-community==0.3.*" "langgraph==0.3.*" "langchain-huggingface" "huggingface_hub" "sentence-transformers" wikipedia -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path = '/content/drive/MyDrive/LangGraph/'

# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

# example

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

print(memory.load_memory_variables({}))
"""
{'history': "Human: hi!\nAI: what's up?"}
"""

# memory key change
memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

print(memory.load_memory_variables({}))
"""
{'chat_history': "Human: hi!\nAI: what's up?"}
"""

# return in str type or list type
memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

print(memory.load_memory_variables({}))
"""
{'history': [HumanMessage(content='hi!'), AIMessage(content="what's up?")]}
"""

# adding memory

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from operator import itemgetter
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

llm_ep = HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b", task="text-generation")
model = ChatHuggingFace(llm=llm_ep)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

inputs = {"input": "Hello, My name is Joon"}
response = chain.invoke(inputs)
print(response)
"""
content="Hello Joon! It's nice to meet you. How can I assist you today?"
"""
memory.save_context(inputs, {"output": response.content}) # type: ignore

print(memory.load_memory_variables({}))
"""
{'history': [HumanMessage(content='Hello, My name is Joon'), AIMessage(content='Hello Joon! How can I assist you today?')]}
"""

inputs = {"input": "whats my name"}
response = chain.invoke(inputs)
print(response)
"""
content='Your name is Joon. How can I assist you further, Joon?
"""

# Conversation Buffer Window Memory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

print(memory.load_memory_variables({}))
"""
{'history': 'Human: not much you\nAI: not much'}
"""

memory = ConversationBufferWindowMemory(k=1, return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

print(memory.load_memory_variables({}))
"""
{'history': [HumanMessage(content='not much you'), AIMessage(content='not much')]}
"""


# entity

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_openai import OpenAI
from langchain.memory import ConversationEntityMemory

llm = OpenAI(temperature=0)
memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

print(memory.load_memory_variables({"input": 'who is Sam'}))
"""
{'history': 'Human: Deven & Sam are working on a hackathon project\n
    AI:  That sounds like a great project! What kind of project are they working on?',
'entities': {'Sam': 'Sam is working on a hackathon project with Deven.'}}
"""

print(memory.load_memory_variables({"input": 'who is Deven'}))
"""
{'history': 'Human: Deven & Sam are working on a hackathon project\n
    AI:  That sounds like a great project! What kind of project are they working on?',
'entities': {
    'Deven': 'Deven is working on a hackathon project with Sam.',
    'Sam': 'Sam is working on a hackathon project with Deven.'}}
"""

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.memory import ConversationEntityMemory

llm_ep = HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b", task="text-generation")
llm = ChatHuggingFace(llm=llm_ep)
memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

print(memory.load_memory_variables({"input": 'who is Sam'}))
"""
{'history': 'Human: Deven & Sam are working on a hackathon project\n
    AI:  That sounds like a great project! What kind of project are they working on?',
'entities': {'Sam': 'Sam is working on a hackathon project with Deven.'}}
"""

print(memory.load_memory_variables({"input": 'who is Deven'}))
"""
{'history': 'Human: Deven & Sam are working on a hackathon project\n
    AI:  That sounds like a great project! What kind of project are they working on?',
'entities': {
    'Deven': 'Deven is working on a hackathon project with Sam.',
    'Sam': 'Sam is working on a hackathon project with Deven.'}}
"""

# Connversation Summary Memory

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI

memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "hi"}, {"output": "whats up"})

print(memory.load_memory_variables({}))
"""
{'history': '\nThe human greets the AI, to which the AI responds.'}
"""

memory = ConversationSummaryMemory(llm=OpenAI(temperature=0), return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})

# print(memory.load_memory_variables({}))
"""
{'history': [SystemMessage(content='\nThe human greets the AI, to which the AI responds.')]}
"""

# Directly use the memory
messages = memory.chat_memory.messages
"""
[HumanMessage(content='hi'), AIMessage(content='whats up')]
"""
previous_summary = ""
print(memory.predict_new_summary(messages, previous_summary))
"""
The human greets the AI, to which the AI responds.
"""

# Initializing with messages/existing summary
history = ChatMessageHistory()
history.add_user_message("hi")
history.add_ai_message("hi there!")

memory = ConversationSummaryMemory.from_messages(
    llm=OpenAI(temperature=0),
    chat_memory=history,
    return_messages=True
)
print(memory.buffer)
"""
The human greets the AI, to which the AI responds with a friendly greeting.
"""

# summary initialization
memory = ConversationSummaryMemory(
    llm=OpenAI(temperature=0),
    buffer="The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.",
    chat_memory=history,
    return_messages=True
)
print(memory.buffer)
"""
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
"""

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm_ep = HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b", task="text-generation")
model = ChatHuggingFace(llm=llm_ep)

memory = ConversationSummaryMemory(llm=model)
memory.save_context({"input": "hi"}, {"output": "whats up"})

print(memory.load_memory_variables({}))
"""
{'history': '\nThe human greets the AI, to which the AI responds.'}
"""

memory = ConversationSummaryMemory(llm=model, return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})

# print(memory.load_memory_variables({}))
"""
{'history': [SystemMessage(content='\nThe human greets the AI, to which the AI responds.')]}
"""

# Directly use the memory
messages = memory.chat_memory.messages
"""
[HumanMessage(content='hi'), AIMessage(content='whats up')]
"""
previous_summary = ""
print(memory.predict_new_summary(messages, previous_summary))
"""
The human greets the AI, to which the AI responds.
"""

# Initializing with messages/existing summary
history = ChatMessageHistory()
history.add_user_message("hi")
history.add_ai_message("hi there!")

memory = ConversationSummaryMemory.from_messages(
    llm=model,
    chat_memory=history,
    return_messages=True
)
print(memory.buffer)
"""
The human greets the AI, to which the AI responds with a friendly greeting.
"""

# summary initialization
memory = ConversationSummaryMemory(
    llm=model,
    buffer="The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.",
    chat_memory=history,
    return_messages=True
)
print(memory.buffer)
"""
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
"""

# Conversation Summary Buffer Memory

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_openai import OpenAI
from langchain.memory import ConversationSummaryBufferMemory

llm = OpenAI()

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

print(memory.load_memory_variables({}))
"""
{'history': "System: \nThe human greets the AI and the AI responds asking what's up.\nHuman: not much you\nAI: not much"}
"""

memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=10, return_messages=True
)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

print(memory.load_memory_variables({}))
"""
{'history': [
    SystemMessage(content="\nThe human greets the AI. The AI responds by asking what's up."),
    HumanMessage(content='not much you'),
    AIMessage(content='not much')]}
"""

messages = memory.chat_memory.messages
previous_summary = ""
print(memory.predict_new_summary(messages, previous_summary))
"""
The human and AI exchange pleasantries and express that there is not much going on.
"""

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.memory import ConversationSummaryBufferMemory

llm_ep = HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b", task="text-generation")
model = ChatHuggingFace(llm=llm_ep)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

print(memory.load_memory_variables({}))
"""
{'history': "System: \nThe human greets the AI and the AI responds asking what's up.\nHuman: not much you\nAI: not much"}
"""

memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=10, return_messages=True
)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

print(memory.load_memory_variables({}))
"""
{'history': [
    SystemMessage(content="\nThe human greets the AI. The AI responds by asking what's up."),
    HumanMessage(content='not much you'),
    AIMessage(content='not much')]}
"""

messages = memory.chat_memory.messages
previous_summary = ""
print(memory.predict_new_summary(messages, previous_summary))
"""
The human and AI exchange pleasantries and express that there is not much going on.
"""